In [84]:

import os
import json
import pandas as pd
import numpy as np
import pickle as pkl
import seaborn as sns
import gensim
from gensim.test.utils import common_texts
import torch

import torch.nn as nn
import torch.nn.functional as F
# load config
with open('config.json', 'r') as f:
    config = json.load(f)
cwd = os.getcwd()
os.chdir(config['REPODIR'])
import Utils as U
from Corpus import Corpus
os.chdir(cwd)

from collections import Counter, defaultdict
import itertools


from tqdm.auto import tqdm, trange
from collections import Counter
import random
from torch import optim
from gensim.models import KeyedVectors
from gensim.test.utils import datapath

from torch.utils.tensorboard import SummaryWriter

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")
import pickle as pkl
from sklearn.decomposition import PCA
from sklearn.metrics import f1_score
import nltk
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

torch.set_default_dtype(torch.float32)

from torch.utils.data.dataloader import default_collate
from torch.utils.tensorboard import SummaryWriter
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
kwargs = {'num_workers': 4, 'pin_memory': True} \
         if (device == "cuda:0" or device == 'mps') else {}
collate_func = lambda x: tuple(x_.to(device) for x_ in default_collate(x)) \
               if device != "cpu" else default_collate

# with open('embedding_data.pkl', 'rb') as f:
#     embed = pkl.load(f)

# embed_df = pd.DataFrame(embed)


# n_classes = embed_df.author_id.nunique()

# from sklearn.preprocessing import OneHotEncoder
# label_encoder=OneHotEncoder(sparse_output=False)

# # -


# X = embed_df['vectors'] # Word Embeddings

# +
test_size = 0.2
val_size = 0.2
random_state =699


# +
with open('sentence_embed.pkl', 'rb') as f:
    embed = pkl.load(f)

embed_df = pd.DataFrame(embed)

data = U.load_file('data_vFFFF.pkl', 'pkl', config['DATADIR'])


In [85]:
embed_df[2][0].shape

(100,)

In [86]:

embed_df = pd.DataFrame(data)

embed_df['join_text'] = embed_df['text'].apply(' '.join)
embed_df = embed_df[['author_id', 'text', 'join_text', 'passage_key']]


In [87]:
import torch
torch.cuda.empty_cache()

In [88]:

n_classes = embed_df.author_id.nunique()

from sklearn.preprocessing import LabelEncoder
label_encoder=LabelEncoder()
y= label_encoder.fit_transform(embed_df['author_id'].to_numpy(dtype='int32').reshape(-1,1))
X = embed_df['passage_key']

X_train, X_test, y_train, y_test = U.train_test_split(X, y, test_size=test_size,
                                                        random_state=random_state,
                                                        stratify=y)

# Split train set into train and validation sets
X_train, X_val, y_train, y_val = U.train_test_split(X_train, y_train, test_size=val_size/(1-test_size),
                                                    random_state=random_state,
                                                    stratify=y_train)



/home/srtoner/.local/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [89]:
embed_df['label'] = y

train_set = set(X_train)
val_set = set(X_val)
test_set = set(X_test)

In [90]:
train = embed_df[embed_df['passage_key'].apply(lambda S: S in train_set)]
val = embed_df[embed_df['passage_key'].apply(lambda S: S in val_set)]
test = embed_df[embed_df['passage_key'].apply(lambda S: S in test_set)]

In [91]:
train.to_csv('train.csv', index=False)
val.to_csv('val.csv', index=False)
test.to_csv('test.csv', index=False)

In [92]:
train

,author_id,text,join_text,passage_key,label
0,2210,"[“No.”, , “I will buy it of you.”, , “No.”, , ...",“No.” “I will buy it of you.” “No.” Phileas...,PG103_7736,18
1,2210,"[Bombay, for which they were now detained at C...","Bombay, for which they were now detained at Ca...",PG103_3686,18
2,2210,"[, “Well, Monsieur Fix,” said Passepartout, “h...","“Well, Monsieur Fix,” said Passepartout, “hav...",PG103_4386,18
3,2210,"[but the intractable Fogg, as reserved as ever...","but the intractable Fogg, as reserved as ever,...",PG103_3836,18
4,2210,"[“I am he.”, , “Is this man your servant?” add...",“I am he.” “Is this man your servant?” added ...,PG103_3536,18
...,...,...,...,...,...
27780,1306,"[ ""He was our playmate; us he understood, ...","""He was our playmate; us he understood ...",PG47265_447,12
27781,1306,"[, , *THE SANDHILLS*, , , Oh, naked-footed...","*THE SANDHILLS* Oh, naked-footed boy, ...",PG47265_397,12
27783,1306,"[*SCANDAL*, , , An owl alighted in the yew...",*SCANDAL* An owl alighted in the yew ...,PG47265_797,12
27784,1306,"[ And by the lattice climbs a crimson rose,...","And by the lattice climbs a crimson rose, ...",PG47265_997,12


In [93]:
from datasets import Dataset

In [94]:
ds = {
      'train' :  Dataset.from_csv('train.csv'),
      'val' :  Dataset.from_csv('val.csv'),
      'test' :  Dataset.from_csv('test.csv'),
      }

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-2d49eeb56c047ef0/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-2847c6b0d9638046/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /home/srtoner/.cache/huggingface/datasets/csv/default-e7504e7417ecc489/0.0.0. Subsequent calls will reuse this data.


In [95]:
embed_df.columns

Index(['author_id', 'text', 'join_text', 'passage_key', 'label'], dtype='object')

In [96]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
from transformers.integrations import TensorBoardCallback
from transformers.trainer_callback import EarlyStoppingCallback
from torch.utils.data import DataLoader


ds1 = {}

BASE_MODEL = "microsoft/MiniLM-L12-H384-uncased"
# BASE_MODEL = "allenai/longformer-base-4096"
# BASE_MODEL = "lreN5bs16" # Learning Rate 2e-5, batch size 16
LEARNING_RATE = 2e-5
MAX_LENGTH = 510
BATCH_SIZE = 16
EPOCHS = 20

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, 
                                                           num_labels=n_classes,
                                                           ignore_mismatched_sizes=True)

def preprocess_function(examples, test = False):
    # if not test:
    label = examples["label"] 
    examples = tokenizer(examples["join_text"],
                        truncation=True, 
                        padding="max_length",
                        max_length=MAX_LENGTH,
                        return_tensors='pt')
    
    for key in examples:
        examples[key] = examples[key].squeeze(0)
  
    # if not test:
    examples["label"] = torch.IntTensor([label])
    examples = examples.to(device)
    return examples

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [97]:
for split in ds:
    ds1[split] = ds[split].map(preprocess_function, 
                                remove_columns=['author_id', 'text', 'passage_key','join_text', 'label'])

    ds1[split].set_format('pt')

Map:   0%|          | 0/16671 [00:00<?, ? examples/s]

Map:   0%|          | 0/5558 [00:00<?, ? examples/s]

Map:   0%|          | 0/5558 [00:00<?, ? examples/s]

In [98]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="output/",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_steps = 1,
    num_train_epochs=EPOCHS,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    metric_for_best_model="F1",
    greater_is_better=True,
    load_best_model_at_end=True,
    weight_decay=0.01,
    report_to = 'tensorboard'
)

early_stop = EarlyStoppingCallback(1, 0.01)
tb = TensorBoardCallback()


from transformers import Trainer
class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        loss_fct = nn.functional.cross_entropy
        loss = loss_fct(logits.view(-1, n_classes), labels.view(-1))
        return (loss, outputs) if return_outputs else loss
    

In [99]:
from sklearn.metrics import f1_score as f1s

def compute_metrics_for_classification(eval_pred):
    predictions, labels = eval_pred
    labels = labels.reshape(-1, 1)
    print(labels)
    print(type(predictions))
    
    predicted_class = predictions.argmax(axis=1)
    print(predicted_class)
    f1 = f1s(labels, predicted_class,average = 'weighted')
    print(f"F1: {f1}")
    
    return {"F1": f1}

In [100]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=ds1["train"],
    eval_dataset=ds1["val"],
    compute_metrics=compute_metrics_for_classification,
)

In [ ]:
trainer.train()

/home/srtoner/.local/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,F1
1,2.006500,2.639907,0.301173
2,2.540000,2.156227,0.360174
3,1.561000,1.832863,0.457310
4,1.539200,1.512801,0.579973
5,1.250000,1.332440,0.628858
6,1.351800,1.196575,0.662417
7,0.401200,1.034151,0.704973
8,0.701600,0.973756,0.712966


[[18]
 [18]
 [18]
 ...
 [12]
 [12]
 [12]]
<class 'numpy.ndarray'>
[17 17 17 ...  1  1 27]
F1: 0.3011732932745492
[[18]
 [18]
 [18]
 ...
 [12]
 [12]
 [12]]
<class 'numpy.ndarray'>
[ 4 17 17 ...  1  1  1]
F1: 0.36017410132429845
[[18]
 [18]
 [18]
 ...
 [12]
 [12]
 [12]]
<class 'numpy.ndarray'>
[ 4 18 18 ... 15 27 27]
F1: 0.45730994628148447
[[18]
 [18]
 [18]
 ...
 [12]
 [12]
 [12]]
<class 'numpy.ndarray'>
[18 18 18 ... 15 27  6]
F1: 0.5799726447943793
[[18]
 [18]
 [18]
 ...
 [12]
 [12]
 [12]]
<class 'numpy.ndarray'>
[18 18 18 ... 50 51 51]
F1: 0.6288581267231885
[[18]
 [18]
 [18]
 ...
 [12]
 [12]
 [12]]
<class 'numpy.ndarray'>
[18 18 18 ... 50 30 10]
F1: 0.6624167732544448
[[18]
 [18]
 [18]
 ...
 [12]
 [12]
 [12]]
<class 'numpy.ndarray'>
[18 18 18 ... 48 10 41]
F1: 0.7049730550835361
[[18]
 [18]
 [18]
 ...
 [12]
 [12]
 [12]]
<class 'numpy.ndarray'>
[18 18 18 ... 48 10 49]
F1: 0.7129659038883883


In [54]:
trainer.evaluate()

[[18]
 [18]
 [18]
 ...
 [12]
 [12]
 [12]]
<class 'numpy.ndarray'>
[18 18 18 ... 49 49 48]
F1: 0.7140582871448018


{'eval_loss': 1.0358072519302368,
 'eval_F1': 0.7140582871448018,
 'eval_runtime': 25.1177,
 'eval_samples_per_second': 221.278,
 'eval_steps_per_second': 13.855,
 'epoch': 10.0}

In [57]:
n_classes

56

prediction = trainer.predict(ds['validation'])

In [58]:
trainer.save_model('hf_model_2')